In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as BS
from datetime import datetime

import pandas as pd
import time
import sys, os
import re
import json


def find_css(css_selector, browser):
    return browser.find_element(By.CSS_SELECTOR, css_selector)
def finds_css(css_selector, browser):
    return browser.find_elements(By.CSS_SELECTOR, css_selector)

def find_xpath(xpath, browser):
    return browser.find_element(By.XPATH, xpath)
def finds_xpath(xpath, browser):
    return browser.find_elements(By.XPATH, xpath)

def find_id(e_id, browser):
    return browser.find_element(By.ID, e_id)

def find_className(cn, browser):
    return browser.find_element(By.CLASS_NAME, cn)
def finds_className(cn , browser):
    return browser.find_elements(By.CLASS_NAME, cn)

def find_linktext(lt, browser):
    return browser.find_element(By.LINK_TEXT, lt)

def find_name(name, browser):
    return browser.find_element(By.NAME, name)
def finds_name(name, browser):
    return browser.find_elements(By.NAME, name)

def find_tagName(tag_name, browser):
    return browser.find_element(By.TAG_NAME, tag_name)

def finds_tagName(tag_name, browser):
    return browser.find_elements(By.TAG_NAME, tag_name)

def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    base_path = getattr(sys, '_MEIPASS', os.path.dirname(os.path.abspath(__file__)))
    return os.path.join(base_path, relative_path)

In [2]:
def open_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--no--sandbox')
    options.add_argument('no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1080,800')
    options.add_argument('incognito')
    chrome_service = Service('chromedriver')
    chrome_service = Service(executable_path="chromedriver.exe")
    browser = webdriver.Chrome(service=chrome_service, options=options)
    
    return browser

In [3]:
def rm_login(rm_id, rm_pw, browser):
    browser.get("https://www.rm1.co.kr/")
    
    try:
        browser.switch_to.frame(0)
    except Exception as ex:
        print(ex)
        pass
    
    time.sleep(.5)
    find_id("stay_sign", browser).click()
    time.sleep(.5)
    try:
        alert = browser.switch_to.alert
        alert.accept()
    except Exception as ex:
        print(ex)
        pass
    
    find_id('user_id', browser).send_keys(rm_id)
    time.sleep(.5)
    find_id('user_pw', browser).send_keys(rm_pw)
    time.sleep(.5)
    find_id('user_pw', browser).send_keys("\n")
    
    time.sleep(2)
    
    try:
        find_id('closeToday', browser).click()
        time.sleep(.3)
        find_id('closeBtn', browser).click()
    except:
        pass

In [4]:
def first_search_company(business_number, browser):
    try:
        search_input = find_id('searchTxt', browser)
        search_input.clear()
        search_input.send_keys(business_number)
        time.sleep(1)
        search_input.send_keys('\n')

        time.sleep(1)
    except:
        browser.get("https://www.rm1.co.kr/")
        try:
            browser.switch_to.frame(0)
        except Exception as ex:
            print(ex)
            pass
        
        time.sleep(1)
        
        search_input = find_id('searchTxt', browser)
        search_input.clear()
        search_input.send_keys(business_number)
        time.sleep(1)
        search_input.send_keys('\n')

        time.sleep(1)

In [5]:
def extract_company_info(browser):
    # 첫번째 나온 회사 링크 클릭
    first_element = find_id('schResultCmpName', browser)
    find_tagName('a', first_element).click()

    time.sleep(3)
    
    soup = BS(browser.page_source, "html.parser")
    soup = soup.find(class_="tbl_l_a mt5")

    company_data = {}

    keys_to_extract = ['기업명', '대표자명', '사업자번호', '종업원수']

    for row in soup.select('table tbody tr'):
        th = row.find('th', scope='row')
        td = row.find('td').get_text(strip=True) if row.find('td') else None
        if th:
            key = th.get_text(strip=True)
            if key in keys_to_extract:
                company_data[key] = td

#     json_data = json.dumps(company_data, ensure_ascii=False, indent=4)
    
    return company_data

In [6]:
def extract_financial_statements(browser, company_data):
    # 재무상태표 click
    find_id('fsdtset1', browser).click()
    
    time.sleep(1)

    financial_statements = {}
    financial_statements_name = []

    soup = BS(browser.page_source, "html.parser")
    table = soup.find(id='fin_a')
    header = table.find('thead').find_all('th')[1:]

    for th in header:
        date = th.get_text(strip=True)
        financial_statements[date] = []

    rows = table.find('tbody').find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        for i, column in enumerate(columns):
            date = list(financial_statements.keys())[i]
            value = column.get_text(strip=True)
            financial_statements[date].append(value)
    
    try:
        financial_statements['2022-12-31(결산)'].insert(0, company_data['기업명'])
        financial_statements['2022-12-31(결산)'].insert(0, company_data['사업자번호'])
        financial_statements_name.insert(0, '회사이름')
        financial_statements_name.insert(1, '사업자번호')
    except:
        pass
    
    if '금액' in financial_statements:
        del financial_statements['금액']

    # print(json.dumps(financial_statements, ensure_ascii=False, indent=4))

    resoup = soup.find(class_="td_dp01")
    th_col_name = resoup.find_all('th')
    for col in th_col_name:
        financial_statements_name.append(col.text)

    return financial_statements_name, financial_statements

In [7]:
def extract_income_statement(browser, company_data):
    find_id('li12', browser).click()
    
    income_statement_value = {}
    income_statement_col_name = []

    soup = BS(browser.page_source, "html.parser")
    table = soup.find(id='fin_a')
    header = table.find('thead').find_all('th')[1:]

    for th in header:
        date = th.get_text(strip=True)
        income_statement_value[date] = []

    rows = table.find('tbody').find_all('tr')
    for row in rows:
        columns = row.find_all('td')
        for i, column in enumerate(columns):
            date = list(income_statement_value.keys())[i]
            value = column.get_text(strip=True)
            income_statement_value[date].append(value)
    
    try:
        income_statement_value['2022-12-31(결산)'].insert(0, company_data['기업명'])
        income_statement_value['2022-12-31(결산)'].insert(1, company_data['사업자번호'])
        income_statement_col_name.insert(0, '회사이름')
        income_statement_col_name.insert(1, '사업자번호')
    except:
        pass
    
    if '금액' in income_statement_value:
        del income_statement_value['금액']

    # print(json.dumps(financial_statements, ensure_ascii=False, indent=4))

    resoup = soup.find(class_="td_dp01")
    th_col_name = resoup.find_all('th')
    for col in th_col_name:
        income_statement_col_name.append(col.text)
    
    return income_statement_col_name, income_statement_value

In [8]:
xls = pd.ExcelFile('회사사업자번호.xlsx')
df = pd.read_excel(xls, '상위')
print(df.shape)
df['사업자번호'] = df['사업자번호'].astype('str')
print("중복치 확인 수 : ",df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.shape)

df_1 = pd.read_excel(xls, '전수')
print(df_1.shape)
df_1['사업자번호'] = df_1['사업자번호'].astype('str')
print("중복치 확인 수 : ",df_1.duplicated().sum())
df_1.drop_duplicates(inplace=True)
print(df_1.shape)

df_2 = pd.read_excel(xls, '표본')
print(df_2.shape)
df_2['사업자번호'] = df_2['사업자번호'].astype('str')
print("중복치 확인 수 : ",df_2.duplicated().sum())
df_2.drop_duplicates(inplace=True)
print(df_2.shape)

FileNotFoundError: [Errno 2] No such file or directory: '회사사업자번호.xlsx'

In [10]:
df = pd.read_excel("연보\\사업자번호리스트.xlsx")
print(df.shape)

(8818, 1)


In [11]:
count = 1

final_company_data = []

final_financial_statements_data = []
final_income_statement_data = []

final_financial_statements_col_name = []
final_income_statement_col_name = []

browser = open_browser()

rm_login('rlaxodud3006', 'rlaxodud3006@', browser)

for business_number in df['사업자번호']:
    try:
        first_search_company(business_number, browser)

        time.sleep(2)
        try:
            company_data = extract_company_info(browser)
            time.sleep(1)
            final_company_data.append(company_data)
            time.sleep(1)
            financial_statements_name, financial_statements = extract_financial_statements(browser, company_data)
            time.sleep(1)
            income_statement_col_name, income_statement_value = extract_income_statement(browser, company_data)
        except Exception:
            pass

        try:
            if '2022-12-31(결산)' in income_statement_value:
                final_income_statement_data.append(income_statement_value['2022-12-31(결산)'])
                final_income_statement_col_name.append(income_statement_col_name)

            if '2022-12-31(결산)' in financial_statements:
                final_financial_statements_data.append(financial_statements['2022-12-31(결산)'])
                final_financial_statements_col_name.append(financial_statements_name)

        except Exception as e:
            print(f"An exception occurred: {str(e)}")
            pass

        print(count)
        count += 1
        
    except NoSuchElementException:
        print("Element not found. Logging in again.")
        rm_login('rlaxodud3006', 'rlaxodud3006@', browser)
        continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
Element not found. Logging in again.
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
Element not found. Logging in again.
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
25

1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
Element not found. Logging in again.
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
Element not found. Logging in again.
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1

3271
3272
3273
3274
3275
3276
3277
3278
3279
3280
3281
3282
3283
3284
3285
3286
3287
3288
3289
3290
3291
3292
3293
3294
3295
3296
3297
3298
3299
3300
3301
3302
3303
3304
3305
3306
3307
3308
3309
Element not found. Logging in again.
3310
3311
3312
3313
3314
3315
3316
3317
3318
3319
3320
3321
3322
3323
3324
3325
3326
3327
3328
3329
3330
3331
3332
3333
3334
3335
3336
3337
3338
3339
3340
3341
3342
3343
3344
3345
3346
3347
3348
3349
3350
3351
3352
3353
3354
3355
3356
3357
3358
3359
3360
3361
3362
3363
3364
3365
3366
3367
3368
3369
3370
3371
3372
3373
3374
3375
3376
3377
3378
3379
3380
3381
3382
3383
3384
3385
3386
3387
3388
3389
3390
3391
3392
3393
3394
3395
3396
3397
3398
3399
3400
3401
3402
3403
3404
3405
3406
3407
3408
3409
3410
3411
Element not found. Logging in again.
3412
3413
3414
3415
3416
3417
3418
3419
3420
3421
3422
3423
3424
3425
3426
3427
3428
3429
3430
3431
3432
3433
3434
3435
3436
3437
3438
3439
3440
3441
3442
3443
3444
3445
3446
3447
3448
3449
3450
3451
3452
3453
3454
3455
3

4799
4800
4801
4802
4803
4804
4805
4806
4807
4808
4809
4810
4811
4812
4813
4814
4815
4816
4817
4818
4819
4820
4821
4822
4823
4824
4825
4826
4827
4828
4829
4830
4831
4832
4833
4834
4835
4836
4837
4838
4839
4840
4841
4842
4843
4844
4845
4846
4847
4848
4849
4850
4851
4852
4853
4854
4855
4856
4857
4858
4859
4860
4861
4862
4863
4864
4865
4866
4867
4868
4869
4870
4871
4872
4873
4874
4875
4876
4877
4878
4879
4880
4881
4882
4883
4884
4885
4886
4887
4888
4889
4890
4891
4892
4893
4894
4895
4896
4897
4898
4899
4900
4901
4902
4903
4904
4905
4906
Element not found. Logging in again.
4907
4908
4909
4910
4911
4912
4913
4914
4915
4916
4917
4918
4919
4920
4921
4922
4923
4924
4925
4926
4927
4928
4929
4930
4931
4932
4933
4934
4935
4936
4937
4938
4939
4940
4941
4942
4943
4944
4945
4946
4947
4948
4949
4950
4951
4952
4953
4954
4955
4956
4957
4958
4959
4960
4961
4962
4963
4964
4965
4966
4967
4968
4969
4970
4971
4972
4973
4974
4975
4976
4977
4978
4979
4980
4981
4982
4983
4984
4985
4986
4987
4988
4989
4990
499

6334
6335
6336
6337
6338
6339
6340
6341
6342
6343
Element not found. Logging in again.
6344
6345
6346
6347
6348
6349
6350
6351
6352
6353
6354
6355
6356
6357
6358
6359
6360
6361
6362
6363
6364
6365
6366
6367
6368
6369
6370
6371
6372
6373
6374
6375
6376
6377
6378
6379
6380
6381
6382
6383
6384
6385
6386
6387
6388
6389
6390
6391
6392
6393
6394
6395
6396
6397
6398
6399
6400
6401
6402
6403
6404
6405
6406
6407
6408
6409
6410
6411
6412
6413
6414
6415
6416
6417
6418
6419
6420
6421
6422
6423
6424
6425
6426
6427
6428
6429
6430
6431
6432
6433
6434
6435
6436
6437
6438
6439
6440
6441
6442
6443
6444
6445
6446
Element not found. Logging in again.
6447
6448
6449
6450
6451
6452
6453
6454
6455
6456
6457
6458
6459
6460
6461
6462
6463
6464
6465
6466
6467
6468
6469
6470
6471
6472
6473
6474
6475
6476
6477
6478
6479
6480
6481
6482
6483
6484
6485
6486
6487
6488
6489
6490
6491
6492
6493
6494
6495
6496
6497
6498
6499
6500
6501
6502
6503
6504
6505
6506
6507
6508
6509
6510
6511
6512
6513
6514
6515
6516
6517
6518
6

Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
Element not found. Logging in again.
E

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.125)
Stacktrace:
	GetHandleVerifier [0x00007FF617D182B2+55298]
	(No symbol) [0x00007FF617C85E02]
	(No symbol) [0x00007FF617B405AB]
	(No symbol) [0x00007FF617B20038]
	(No symbol) [0x00007FF617BA6BC7]
	(No symbol) [0x00007FF617BBA15F]
	(No symbol) [0x00007FF617BA1E83]
	(No symbol) [0x00007FF617B7670A]
	(No symbol) [0x00007FF617B77964]
	GetHandleVerifier [0x00007FF618090AAB+3694587]
	GetHandleVerifier [0x00007FF6180E728E+4048862]
	GetHandleVerifier [0x00007FF6180DF173+4015811]
	GetHandleVerifier [0x00007FF617DB47D6+695590]
	(No symbol) [0x00007FF617C90CE8]
	(No symbol) [0x00007FF617C8CF34]
	(No symbol) [0x00007FF617C8D062]
	(No symbol) [0x00007FF617C7D3A3]
	BaseThreadInitThunk [0x00007FFB8CF7257D+29]
	RtlUserThreadStart [0x00007FFB8DFAAA58+40]


In [24]:
business_number

6178151062

In [12]:
# 회사 이름 정보
print(len(final_company_data))

print(len(final_financial_statements_data))
print(len(final_income_statement_data))

print(len(final_financial_statements_col_name))
print(len(final_income_statement_col_name))

6406
3171
3171
3171
3171


In [13]:
df2 = pd.DataFrame(final_financial_statements_data)
df2_1 = pd.DataFrame(final_financial_statements_col_name)
df3 = pd.DataFrame(final_income_statement_data)
df3_1 = pd.DataFrame(final_income_statement_col_name)
df4 = pd.DataFrame(final_company_data)

In [15]:
df2.to_excel('연보\\연보2\\표본_제무제표1_value.xlsx', index=False)
df2_1.to_excel('연보\\연보2\\표본_제무제표1_key.xlsx', index=False)
df3.to_excel('연보\\연보2\\표본_손익계산서1_value.xlsx', index=False)
df3_1.to_excel('연보\\연보2\\표본_손익계산서1_key.xlsx', index=False)
df4.to_excel('연보\\연보2\\회사인원정보1_전수.xlsx', index=False)

In [16]:
# 재무제표
list_of_lists = df2.values.tolist()
list_of_lists_col = df2_1.values.tolist()

final_list = []

for i, j in zip(list_of_lists_col, list_of_lists):
    final_list.append(i)
    final_list.append(j)
    
final_df = pd.DataFrame(final_list)

# final_df.to_excel('재무제표1 예시.xlsx', index=False)

In [17]:
# 손익계산서

list_of_lists1 = df3.values.tolist()
list_of_lists_col1 = df3_1.values.tolist()

final_list1 = []

for i, j in zip(list_of_lists_col1, list_of_lists1):
    final_list1.append(i)
    final_list1.append(j)
    
final_df1 = pd.DataFrame(final_list1)

# final_df1.to_excel('손익계산서 예시.xlsx', index=False)

In [18]:
print(len(list_of_lists1))
print(len(list_of_lists_col1))

3171
3171


In [19]:
# 재무제표 전처리
final_dict1 = []

for i ,j in zip(list_of_lists_col, list_of_lists):
    result2 = dict(zip(i, j))
    final_dict1.append(result2)

# 손익계산서 전처리
final_dict = []

for i ,j in zip(list_of_lists_col1, list_of_lists1):
    result1 = dict(zip(i, j))
    final_dict.append(result1)

In [20]:
final_lists1 = []

for ai in final_dict:
    try:
        filtered_data = {key: ai[key] for key in selected_items}
        final_lists1.append(filtered_data)
    except:
        pass


column_names = [
    '매출액', '매출원가', '판매비와관리비', '급료와임금', '퇴직급여충당금전입액',
    '복리후생비', '세금과공과', '임차료비용', '감가상각비', '경상연구개발비',
    '대손상각비', '영업이익(손실)', '이자수익', '이자비용'
]

result_dict1 = {}

for company_data in final_dict:
    company_name = company_data['회사이름']
    
    company_result = {column_name: 0 for column_name in column_names}
    
    for column_name in column_names:
        if column_name in company_data:
            value = company_data[column_name]
        else:
            value = 0
        
        company_result[column_name] = value
    
    result_dict1[company_name] = company_result
    
print(len(result_dict1))
print(result_dict1)

2987
{'에이원축산(주)': {'매출액': '87,459', '매출원가': 0, '판매비와관리비': '92,236', '급료와임금': '54,000', '퇴직급여충당금전입액': 0, '복리후생비': '6,305', '세금과공과': '2,265', '임차료비용': '3,600', '감가상각비': '448', '경상연구개발비': 0, '대손상각비': '247', '영업이익(손실)': '-4,777', '이자수익': '4', '이자비용': 0}, '(주)해나소프트': {'매출액': '960,784', '매출원가': '-', '판매비와관리비': '914,655', '급료와임금': '218,680', '퇴직급여충당금전입액': 0, '복리후생비': '43,732', '세금과공과': '16,518', '임차료비용': '21,571', '감가상각비': '7,353', '경상연구개발비': 0, '대손상각비': '-', '영업이익(손실)': '46,128', '이자수익': '9,804', '이자비용': '3'}, '(주)미디어온': {'매출액': '637,254', '매출원가': 0, '판매비와관리비': '632,344', '급료와임금': '251,499', '퇴직급여충당금전입액': 0, '복리후생비': '27,388', '세금과공과': '13,603', '임차료비용': '5,012', '감가상각비': '1,523', '경상연구개발비': 0, '대손상각비': 0, '영업이익(손실)': '4,909', '이자수익': '36', '이자비용': '5,447'}, '(주)유월커뮤니케이션': {'매출액': '1,212,288', '매출원가': 0, '판매비와관리비': '1,146,518', '급료와임금': '362,181', '퇴직급여충당금전입액': 0, '복리후생비': '3,165', '세금과공과': '38,831', '임차료비용': '52,667', '감가상각비': '1,453', '경상연구개발비': 0, '대손상각비': 0, '영업이익(손실)': '65,770', '이자수익':

In [21]:
income_df = pd.DataFrame(result_dict1).T
income_df.shape

income_df.to_excel("연보\\연보2\\손익계산서_final_전수.xlsx")

In [22]:
column_names = ['유형자산(계)', '토지', '건물', '구축물', '(구축물감가상각누계액)', '기계장치', '(기계장치감가상각누계액)', '차량운반구', '(차량운반구감가상각누계액)', '공구와기구', '(공구와기구감가상각누계액)', '비품', '(비품감가상각누계액)', '기타유형자산', '(기타유형자산감가상각누계액)', '건설중인자산', '무형자산(계)', '(상각누계액)']
result_dict = {}

for company_data in final_dict1:
    company_name = company_data['회사이름']
    
    company_result = {column_name: 0 for column_name in column_names}
    
    for column_name in column_names:
        if column_name in company_data:
            value = company_data[column_name]
        else:
            value = 0
        
        company_result[column_name] = value
    
    result_dict[company_name] = company_result
    
print(len(result_dict))
print(result_dict)

2996
{'214-87-97686': {'유형자산(계)': '166', '토지': 0, '건물': 0, '구축물': 0, '(구축물감가상각누계액)': 0, '기계장치': 0, '(기계장치감가상각누계액)': 0, '차량운반구': '1', '(차량운반구감가상각누계액)': '8,559', '공구와기구': 0, '(공구와기구감가상각누계액)': 0, '비품': 0, '(비품감가상각누계액)': 0, '기타유형자산': '165', '(기타유형자산감가상각누계액)': '8,722', '건설중인자산': 0, '무형자산(계)': 0, '(상각누계액)': 0}, '201-86-39553': {'유형자산(계)': '49,582', '토지': 0, '건물': 0, '구축물': 0, '(구축물감가상각누계액)': 0, '기계장치': 0, '(기계장치감가상각누계액)': 0, '차량운반구': '16,850', '(차량운반구감가상각누계액)': '1,204', '공구와기구': 0, '(공구와기구감가상각누계액)': 0, '비품': 0, '(비품감가상각누계액)': 0, '기타유형자산': '32,731', '(기타유형자산감가상각누계액)': '49,572', '건설중인자산': 0, '무형자산(계)': '51,614', '(상각누계액)': 0}, '140-81-56970': {'유형자산(계)': '151,989', '토지': 0, '건물': 0, '구축물': 0, '(구축물감가상각누계액)': 0, '기계장치': 0, '(기계장치감가상각누계액)': 0, '차량운반구': '44,156', '(차량운반구감가상각누계액)': '1,523', '공구와기구': 0, '(공구와기구감가상각누계액)': 0, '비품': 0, '(비품감가상각누계액)': 0, '기타유형자산': '107,833', '(기타유형자산감가상각누계액)': '40,892', '건설중인자산': 0, '무형자산(계)': '96,000', '(상각누계액)': 0}, '107-87-86839': {'유형자산(계)': '9,298', '토지': 0, '건물':

In [23]:
income_df = pd.DataFrame(result_dict).T
income_df.shape

income_df.to_excel("연보\\연보2\\재무제표_final_전수.xlsx")